# Classification Dataset

### Data Mining Project 2024/25

Authors: Nicola Emmolo, Simone Marzeddu, Jacopo Raffi

In [3]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import ClusterCentroids
from scipy.stats import ks_2samp

import pandas as pd
import numpy 
from sklearn.model_selection import train_test_split
import seaborn as sb

In [4]:
dataset = pd.read_csv('../../data/complete_dataset_fe.csv', dtype={'cyclist_team': str})
RANDOM_STATE = 42   

In [5]:
dataset['label'] = (dataset['raw_position'] <= 20).astype(int) #1 if top 20, 0 otherwise 

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589739 entries, 0 to 589738
Data columns (total 38 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   cyclist_url                  589739 non-null  object 
 1   cyclist_name                 589739 non-null  object 
 2   birth_year                   589739 non-null  int64  
 3   weight                       480007 non-null  float64
 4   height                       480007 non-null  float64
 5   nationality                  589739 non-null  object 
 6   race_url                     589739 non-null  object 
 7   race_name                    589739 non-null  object 
 8   points                       589739 non-null  float64
 9   uci_points                   422184 non-null  float64
 10  length                       589739 non-null  float64
 11  climb_total                  475338 non-null  float64
 12  profile                      475338 non-null  float64
 13 

Feature da eliminare:

- cyclist_url (non generalizzabili)
- cyclist_name (non generalizzabili)
- birth_year (gia incluso in age)
- nationality (troppo sbilanciamento nel dataset)
- weight, height (gia incluso in BMI)
- uci_points (troppo pochi e gia inclusi in points)
- race_url (non generalizzabili)
- race_name (non generalizzabili)
- climb_total, profile (gia inclusi in race_physical_effort)
- mostly_tarmac (sbilanciato)
- cyclist_team (non generalizzabili)
- date (non generalizzabili)
- raw_position, position (obv)
- delta (obv) 
- race_year (non generalizzabili)
- race_stage (non generalizzabili)
- cyclist_age_group (??)
- cyclist_climb_power (feature generata con delta, obv)
- points, startlist_quality (gia inclusti in race_prestige)

We must choose between cyclist_age and cyclist_age_group as both these features represent the same information bu give different advantage/disadvantage to our classification process: 
- cyclist_age -> lower dimensionality (no one-hot) but unbalanced representation
- cyclist_age_group -> arbitraty encoding but balanced representation 

we chose to delete cyclist_age as the encoding is arbitraty but it still respects the "order" of the age_groups

In [7]:
columns = ['length', 'race_season', 'cyclist_bmi', 'cyclist_age_group', 'climb_percentage', 'race_physical_effort', 'race_prestige', 'race_year', 
           'previous_mean_position', 'previous_mean_delta', 'previous_mean_cp', 'cyclist_previous_experience', 'label']
dataset = dataset[columns].dropna()

In [8]:
development_dataset = dataset[dataset['race_year'] < 2022]
test_dataset = dataset[dataset['race_year'] >= 2022]

development_dataset = development_dataset.drop(columns=['race_year'])
test_set = test_dataset.drop(columns=['race_year'])
test_label = test_set.pop('label')

In [9]:
label = development_dataset.pop('label')
train_set, val_set, train_label, val_label = train_test_split(development_dataset, label, stratify=label, test_size=0.20, random_state=RANDOM_STATE)

In [10]:
development_set = development_dataset.copy()
developmente_label = label

In [11]:
developmente_label.value_counts(normalize=True)

label
0    0.855363
1    0.144637
Name: proportion, dtype: float64

In [12]:
model = RandomUnderSampler(random_state=RANDOM_STATE)
undersampled_data, undersampled_labels = model.fit_resample(development_set, developmente_label)
undersampled_labels = undersampled_labels.to_numpy().reshape(-1, 1)
undersampled_dataset = pd.DataFrame(numpy.hstack((undersampled_data, undersampled_labels)), columns=(list(development_dataset.columns) + ['label']))

In [13]:
development_set['label'] = label
columns = undersampled_dataset.columns
test_results = [
    ks_2samp(
        development_set[column],
        undersampled_dataset[column],
        alternative="two-sided"
    )
    for column in columns
]
test_data = [(
    test.statistic,
    test.pvalue,
    test.statistic_location
    )
    for test in test_results        
]
test_data = pd.DataFrame(test_data, columns=["KS_test", "p_value", "margin"])

test_data.describe()

,KS_test,p_value
count,12.000000,1.200000e+01
mean,0.058161,1.235782e-01
std,0.099669,2.249733e-01
min,0.002836,0.000000e+00
25%,0.007601,5.780301e-311
50%,0.014647,5.589033e-11
75%,0.069625,1.073549e-01
max,0.355363,5.448123e-01


In [14]:
dev_undersample = undersampled_dataset
dev_undersample['label'].value_counts(normalize=True)

label
0    0.5
1    0.5
Name: proportion, dtype: float64

In [15]:
dev_label = dev_undersample.pop('label')

In [16]:
def discretize_data(dataset, variables):
    for variable in variables:
        #get the unique variable's values
        var = sorted(dataset[variable].unique())
        
        #generate a mapping from the variable's values to the number representation  
        mapping = dict(zip(var, range(0, len(var) + 1)))

        #add a new colum with the number representation of the variable
        dataset[variable+'_num'] = dataset[variable].map(mapping).astype(int)
    return dataset

In [17]:
#discretize the variable relationship, sex, occupation, marital-status, native-country, race
variables = ['cyclist_age_group']
dev_undersample = discretize_data(dev_undersample, variables)
#drop the categorical variables since we don't need them anymore 
dev_undersample.drop(columns=variables, axis=1,inplace=True)
cat_columns = ['race_season']
dev_undersample_encoded = pd.get_dummies(dev_undersample, columns = cat_columns, prefix_sep='%')
dev_undersample_encoded

,length,cyclist_bmi,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,cyclist_age_group_num,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
0,186.0,20.811655,0.028586,0.445758,0.108454,0.441503,377.980769,0.019083,0.069235,0,False,True,False,False
1,149.7,22.893074,0.016593,0.083803,0.110119,0.733762,904.612903,0.00055,0.046961,0,False,False,True,False
2,203.0,22.256908,0.009537,0.08857,0.076395,0.702375,740.479167,0.001277,0.268919,2,False,False,False,True
3,203.5,21.061628,0.012629,0.058932,0.225007,0.486413,389.95614,0.002359,0.233411,2,False,False,True,False
4,157.0,21.037253,0.015185,0.126527,0.145459,0.739083,639.848837,0.000697,0.096681,3,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102351,256.0,20.830818,0.006281,0.046385,0.788245,0.593957,523.114286,0.000767,0.093482,0,True,False,False,False
102352,143.0,20.830818,0.012441,0.057332,0.074522,0.590682,509.652778,0.001097,0.130024,1,False,True,False,False
102353,184.0,20.830818,0.010522,0.08028,0.074522,0.590725,520.333333,0.001264,0.136932,1,False,True,False,False
102354,214.0,20.830818,0.011561,0.119316,0.074522,0.586011,511.708861,0.00177,0.144487,1,False,True,False,False


In [18]:
test_set = discretize_data(test_set, variables)
test_set.drop(columns=variables, axis=1,inplace=True)
cat_columns = ['race_season']
test_set_encoded = pd.get_dummies(test_set, columns = cat_columns, prefix_sep='%')
test_set_encoded

,length,cyclist_bmi,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,cyclist_age_group_num,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
201,203.9,21.705129,0.009858,0.092363,0.225487,0.729362,470.333333,0.000047,0.009954,3,False,False,False,True
202,254.1,21.705129,0.013617,0.198139,0.248896,0.666310,458.285714,0.000047,0.012297,3,False,True,False,False
204,204.2,20.756387,0.008159,0.076668,0.264741,0.906142,591.500000,0.000036,0.002329,0,False,False,False,True
205,203.9,20.756387,0.009858,0.092363,0.225487,0.876822,643.666667,0.000031,0.003379,0,False,False,False,True
206,183.7,20.756387,0.008977,0.068267,0.228368,0.828177,559.000000,0.000032,0.006341,0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589373,154.0,20.305175,0.010779,0.028805,0.078556,0.420047,468.336898,0.002282,0.600797,4,False,False,False,True
589374,294.0,20.305175,0.007327,0.142718,0.455848,0.419732,467.264000,0.002279,0.611609,4,False,False,False,True
589375,204.1,20.305175,0.009882,0.092776,0.304715,0.420876,467.380319,0.002274,0.746805,4,False,False,False,True
589376,273.4,20.305175,0.007992,0.134628,0.399068,0.421826,468.395225,0.002271,0.604920,4,False,True,False,False


Abbiamo trasformato gli attributi categorici in equivalenti numerici per poter essere processati dai modelli di learning: 
- one-hot encoding: è stato utilizzato quando l'attributo categorico non è di tipo ordinale, in questo caso non siamo interessati a preservare ordine o relazioni di vicinanza tra i valori di questo attributo e pertanto vogliamo massimizzare la dissimilarità tra le classi
- discretizzazione: abbiamo usato questa tecnica solo nel caso dell'attributo cyclist_age_group, in quanto queste classi possiedono relazioni di ordine e vicinanza (attributo ordinale)

In [19]:
dev_undersample_encoded['label'] = dev_label
test_set_encoded['label'] = test_label

In [21]:
dev_undersample_encoded.to_csv('../../data/ml_datasets/undersampling/dev_set.csv', index=False)
test_set_encoded.to_csv('../../data/ml_datasets/undersampling/test_set.csv', index=False)